In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
!mkdir data
!cd data
!kaggle datasets download -d parisrohan/credit-score-classification
!unzip -f credit-score-classification.zip

In [ ]:
train_df = pd.read_csv("data/train.csv", low_memory=False)
test_df = pd.read_csv("data/test.csv", low_memory=False)
train_df

# EDA

Getting just a base information about our dataset (types of columns, how much NaNs we have etc)

In [ ]:
train_df.info()

Prepare separate variables for different column types:

In [ ]:
number_columns = train_df.select_dtypes("number").columns
categorical_columns = train_df.select_dtypes("object").columns.drop("Credit_Score")
feature_columns = categorical_columns.union(number_columns)
target_column = "Credit_Score"
number_columns, categorical_columns

Finding nulls in data:

In [ ]:
train_df.isnull().sum()

`Name` column is not important (as ID columns), so we can drop them. Let's create auxillary function to delete column from Index variable and from Dataframe:

In [ ]:
def delete_col(cols):
    global train_df, categorical_columns, number_columns
    train_df.drop(columns=cols, inplace=True)
    
    for col in cols:
        if col in categorical_columns:
            categorical_columns = categorical_columns.drop(col)
        else:
            number_columns = number_columns.drop(col)

delete_col(["Name", "ID", "Customer_ID"])

Although usage of `global` is not recommended, in this case it is okay.

In [ ]:
pd.get_dummies(train_df, columns=categorical_columns, drop_first=True)

In [ ]:
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer()

knn_imputer.fit_transform(train_df)

In [ ]:
train_df.describe()

In [ ]:
import seaborn as sns



# Baseline

In [ ]:
from sklearn.dummy import DummyClassifier

# dummy_classifier = DummyClassifier(random_state=42)

# dummy_classifier.fit-